# Volume - Single Volume Operations

A Volume represents a single 3D or 4D radiology acquisition backed by TileDB. This notebook covers:

- Volume properties and metadata
- **Partial reads** (the key efficiency feature)
- Orthogonal slices: `axial()`, `sagittal()`, `coronal()`
- Arbitrary ROI: `slice()`
- Full volume: `to_numpy()`
- **Analysis methods**: `get_statistics()`, `compute_histogram()`
- **Export**: `to_nifti()`
- Standalone creation: `from_nifti()`, `from_numpy()`

**Prerequisites:** Run [00_ingest_brats.ipynb](./00_ingest_brats.ipynb) first to create the RadiObject.

## Setup

In [ ]:
import sys
sys.path.insert(0, '..')

import tempfile
import shutil
from pathlib import Path

import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt

from config import BRATS_URI, S3_REGION
from src.radi_object import RadiObject
from src.volume import Volume
from src.ctx import configure, S3Config, TileConfig, SliceOrientation, CompressionConfig, Compressor

print(f"RadiObject URI: {BRATS_URI}")

In [ ]:
# Configure S3 if using S3 URI
if BRATS_URI.startswith("s3://"):
    configure(s3=S3Config(region=S3_REGION))

configure(
    tile=TileConfig(orientation=SliceOrientation.AXIAL),
    compression=CompressionConfig(algorithm=Compressor.ZSTD, level=3)
)

## Load RadiObject from URI

In [ ]:
radi = RadiObject(BRATS_URI)
radi

## Getting a Volume

Access volumes through collections. No data is loaded until you request it.

In [ ]:
# Get a volume (lazy - no data loaded yet)
vol = radi.FLAIR.iloc[0]
vol

## Volume Properties

In [ ]:
print(f"Shape: {vol.shape}")
print(f"Dimensions: {vol.ndim}D")
print(f"Data type: {vol.dtype}")
print(f"Observation ID: {vol.obs_id}")

In [ ]:
# Orientation metadata
orient_info = vol.orientation_info
if orient_info:
    print(f"Axis codes: {orient_info.axcodes}")
    print(f"Is canonical (RAS): {orient_info.is_canonical}")
    print(f"Source: {orient_info.source}")

## Partial Reads - The Key Efficiency Feature

TileDB enables reading only the data you need, without loading the entire volume into memory. This is critical for:

- **Large datasets** that don't fit in RAM
- **Quick previews** and exploratory analysis
- **Slice-by-slice processing** pipelines

### Orthogonal Slices: `axial()`, `sagittal()`, `coronal()`

In [ ]:
# Partial reads - only the requested slice is loaded
axial = vol.axial(z=77)        # X-Y plane at Z=77
sagittal = vol.sagittal(x=120)  # Y-Z plane at X=120
coronal = vol.coronal(y=120)    # X-Z plane at Y=120

print(f"Axial shape: {axial.shape}")
print(f"Sagittal shape: {sagittal.shape}")
print(f"Coronal shape: {coronal.shape}")

In [ ]:
# Visualize orthogonal views
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

axes[0].imshow(axial.T, cmap='gray', origin='lower')
axes[0].set_title('Axial (Z=77)')
axes[0].set_xlabel('X'); axes[0].set_ylabel('Y')

axes[1].imshow(sagittal.T, cmap='gray', origin='lower')
axes[1].set_title('Sagittal (X=120)')
axes[1].set_xlabel('Y'); axes[1].set_ylabel('Z')

axes[2].imshow(coronal.T, cmap='gray', origin='lower')
axes[2].set_title('Coronal (Y=120)')
axes[2].set_xlabel('X'); axes[2].set_ylabel('Z')

plt.suptitle('Partial Reads - Only Requested Slices Loaded')
plt.tight_layout()
plt.show()

### Arbitrary ROI: `slice()`

Read any 3D region of interest.

In [ ]:
# Read a 3D ROI (e.g., tumor region)
roi = vol.slice(
    x=slice(80, 160),   # X: 80-160
    y=slice(80, 160),   # Y: 80-160
    z=slice(60, 100)    # Z: 60-100
)

print(f"ROI shape: {roi.shape}")

In [ ]:
# Visualize the ROI
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

mid_z, mid_x, mid_y = roi.shape[2]//2, roi.shape[0]//2, roi.shape[1]//2

axes[0].imshow(roi[:, :, mid_z].T, cmap='gray', origin='lower')
axes[0].set_title('ROI Axial')

axes[1].imshow(roi[mid_x, :, :].T, cmap='gray', origin='lower')
axes[1].set_title('ROI Sagittal')

axes[2].imshow(roi[:, mid_y, :].T, cmap='gray', origin='lower')
axes[2].set_title('ROI Coronal')

plt.suptitle('3D ROI Partial Read')
plt.tight_layout()
plt.show()

### Memory Comparison

In [ ]:
# Compare memory usage: partial vs full read
full_size = np.prod(vol.shape) * 4  # float32 = 4 bytes
roi_size = roi.nbytes
slice_size = axial.nbytes

print(f"Full volume: {full_size / 1024:.1f} KB")
print(f"3D ROI (80x80x40): {roi_size / 1024:.1f} KB ({full_size/roi_size:.1f}x smaller)")
print(f"Single slice (240x240): {slice_size / 1024:.1f} KB ({full_size/slice_size:.1f}x smaller)")

## Full Volume: `to_numpy()`

When you need the complete volume (e.g., for ML training or whole-volume analysis).

In [ ]:
# Load full volume into memory
full_data = vol.to_numpy()

print(f"Shape: {full_data.shape}")
print(f"Memory: {full_data.nbytes / (1024*1024):.2f} MB")
print(f"\nIntensity statistics:")
print(f"  Min: {full_data.min():.2f}")
print(f"  Max: {full_data.max():.2f}")
print(f"  Mean: {full_data.mean():.2f}")
print(f"  Std: {full_data.std():.2f}")

## Analysis Methods

### Statistics: `get_statistics()`

In [ ]:
# Compute descriptive statistics
stats = vol.get_statistics(percentiles=[5, 25, 75, 95])

print("Volume Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value:.2f}")

### Histogram: `compute_histogram()`

In [ ]:
# Compute and visualize histogram
hist_counts, bin_edges = vol.compute_histogram(bins=100)

fig, ax = plt.subplots(figsize=(10, 4))
ax.bar(bin_edges[:-1], hist_counts, width=np.diff(bin_edges), align='edge', alpha=0.7)
ax.set_xlabel('Intensity')
ax.set_ylabel('Count')
ax.set_title('Volume Intensity Histogram')
ax.set_xlim(0, stats['p95'] * 1.2)  # Focus on main distribution
plt.tight_layout()
plt.show()

### Export: `to_nifti()`

In [ ]:
# Export volume to NIfTI
TEMP_DIR = tempfile.mkdtemp(prefix="vol_demo_")
export_path = Path(TEMP_DIR) / "exported_flair.nii.gz"
vol.to_nifti(export_path)

# Verify export
exported = nib.load(export_path)
print(f"Exported to: {export_path}")
print(f"Shape: {exported.shape}")
print(f"Data matches: {np.allclose(vol.to_numpy(), np.asarray(exported.dataobj))}")

# Cleanup
shutil.rmtree(TEMP_DIR)

## Visualization Patterns

In [ ]:
# Axial slice montage
n_slices = 9
z_indices = np.linspace(30, 120, n_slices, dtype=int)

fig, axes = plt.subplots(3, 3, figsize=(10, 10))
for ax, z in zip(axes.flat, z_indices):
    ax.imshow(vol.axial(z).T, cmap='gray', origin='lower')
    ax.set_title(f'Z={z}')
    ax.axis('off')

plt.suptitle('Axial Slice Montage (Partial Reads)')
plt.tight_layout()
plt.show()

In [ ]:
# Compare modalities at same location
subject_ids = radi.obs_subject_ids
subject_id = subject_ids[0]
z = 77
modalities = ["FLAIR", "T1w", "T1gd", "T2w"]

fig, axes = plt.subplots(1, 4, figsize=(14, 3.5))
for i, mod in enumerate(modalities):
    vol_mod = radi.collection(mod).loc[f"{subject_id}_{mod}"]
    axes[i].imshow(vol_mod.axial(z).T, cmap='gray', origin='lower')
    axes[i].set_title(mod)
    axes[i].axis('off')

plt.suptitle(f'{subject_id} - All Modalities at Z={z}')
plt.tight_layout()
plt.show()

## Standalone Volume Creation

Create volumes directly without a RadiObject.

In [ ]:
# Create from NumPy array
TEMP_DIR = tempfile.mkdtemp(prefix="vol_demo_")

synthetic_data = np.random.randn(64, 64, 32).astype(np.float32)
vol_from_numpy = Volume.from_numpy(
    uri=str(Path(TEMP_DIR) / "synthetic_vol"),
    data=synthetic_data,
)
print(f"Volume from NumPy: {vol_from_numpy}")

# Cleanup
shutil.rmtree(TEMP_DIR)

In [ ]:
# Example: Creating from NIfTI file
print("""To create a volume from a NIfTI file:

from src.volume import Volume

vol = Volume.from_nifti(
    uri="./my_volume",
    nifti_path="/path/to/scan.nii.gz",
)
""")

## 4D Volumes

Volumes can be 4D (e.g., fMRI time series). The `t` parameter is optional in slice methods.

In [ ]:
# Create a 4D volume (simulated fMRI)
TEMP_DIR = tempfile.mkdtemp(prefix="vol_demo_")

fmri_data = np.random.randn(64, 64, 32, 100).astype(np.float32)
vol_4d = Volume.from_numpy(
    uri=str(Path(TEMP_DIR) / "fmri_vol"),
    data=fmri_data,
)

print(f"4D Volume: {vol_4d}")

# Partial read at specific time point
slice_t10 = vol_4d.axial(z=16, t=10)
print(f"Axial at Z=16, T=10: shape {slice_t10.shape}")

# Cleanup
shutil.rmtree(TEMP_DIR)

## Summary

| Method | Use Case |
|--------|----------|
| `vol.axial(z)` | Quick preview, slice-by-slice viewing |
| `vol.sagittal(x)` | Cross-sectional analysis |
| `vol.coronal(y)` | Frontal plane viewing |
| `vol.slice(x, y, z)` | ROI extraction, region-based analysis |
| `vol.to_numpy()` | ML training, whole-volume processing |
| `vol.get_statistics()` | Descriptive statistics (mean, std, percentiles) |
| `vol.compute_histogram()` | Intensity distribution analysis |
| `vol.to_nifti()` | Export for external tools |

**Key takeaway:** Use partial reads (`axial`, `sagittal`, `coronal`, `slice`) for exploration. Use analysis methods (`get_statistics`, `compute_histogram`) for quantitative assessment. Reserve `to_numpy()` for when you need the full volume.

## Next Steps

- [04_storage_configuration.ipynb](./04_storage_configuration.ipynb) - Optimize tile orientation for your analysis pattern